In [1]:
import pandas as pd

In [2]:
#definindo estados do nordeste e suas siglas referentes
etiquetas = ['AC','AL','AP', 'AM', 'BA-1', 'BA-2','CE','DF','ES','GO','MA','MT','MS','MG-1','MG-2','PA','PB','PR',
             'PE','PI','RJ-1', 'RJ-2','RN','RS-1', 'RS-2','RO','RR','SC-1', 'SC-2', 'SP-1','SP-2', 'SP-3', 'SP-4', 'SP-5', 'SE', 'TO']
ne = ['ALAGOAS', 'BAHIA', 'CEARÁ', 'SERGIPE', 'RIO GRANDE DO NORTE', 'PERNAMBUCO', 'PIAUÍ', 'MARANHÃO', 'PARAÍBA']


In [ ]:
#1st parte dos dados
data_br =pd.DataFrame()
for sigla in etiquetas[0:18]:
    data_br = pd.concat([data_br ,(pd.read_csv('https://s3-sa-east-1.amazonaws.com/ckan.saude.gov.br/dados-{}.csv'.format(sigla.lower()), 
                                               encoding='latin-1', sep=';', low_memory=False,  error_bad_lines=False))], axis=0, ignore_index=True)


In [ ]:
data_br.shape

In [250]:
data_br.estado.unique()

array(['ACRE', 'AMAZONAS', 'BAHIA', 'RIO GRANDE DO SUL', 'PARANÁ',
       'SÃO PAULO', 'PIAUÍ', 'CEARÁ', 'RONDÔNIA', 'MINAS GERAIS', 'GOIÁS',
       'DISTRITO FEDERAL', 'RIO DE JANEIRO', 'MATO GROSSO', 'RORAIMA',
       'PARAÍBA', 'SANTA CATARINA', 'MARANHÃO', 'PARÁ', 'SERGIPE',
       'PERNAMBUCO', 'MATO GROSSO DO SUL', 'ESPÍRITO SANTO',
       'RIO GRANDE DO NORTE', 'ALAGOAS', nan, 'TOCANTINS', 'AMAPÁ'],
      dtype=object)

In [251]:
data_br =  data_br.loc[data_br.estado.isin(ne)]
data_br.estado.unique()


array(['BAHIA', 'PIAUÍ', 'CEARÁ', 'PARAÍBA', 'MARANHÃO', 'SERGIPE',
       'PERNAMBUCO', 'RIO GRANDE DO NORTE', 'ALAGOAS'], dtype=object)

In [252]:
#2st partes dos dados
for sigla in etiquetas[18:36]:
    data_br = pd.concat([data_br ,(pd.read_csv('https://s3-sa-east-1.amazonaws.com/ckan.saude.gov.br/dados-{}.csv'.format(sigla.lower()), encoding='latin-1', sep=';', low_memory=False, error_bad_lines=False))], axis=0, ignore_index=True)
data_br.shape

b'Skipping line 591719: expected 30 fields, saw 32\n'


(14480275, 30)

In [ ]:
data_br =  data_br.loc[data_br.estado.isin(ne)]
data_br.estado.unique()

In [1]:
data_br.shape

NameError: ignored

In [ ]:
#maioria que teve sindrome respiratória foram os técnicos ou auxiliares de enfermagem
data_br.cbo.value_counts()

In [ ]:
data_br.resultadoTeste.unique()

In [ ]:
#casos de covid com resultado positivo e estado dos nordestes
covid_ne = pd.DataFrame()
covid_ne = data_br.loc[data_br.resultadoTeste == 'Positivo']
covid_ne = covid_ne.loc[covid_ne.dataInicioSintomas != 'undefined']
print(covid_ne.estado.unique())
print(covid_ne.resultadoTeste.unique())

In [ ]:
#data_br.shape = (14.392.792, 30)
covid_ne.shape



In [ ]:
covid_ne.columns

In [ ]:
#selecionando colunas que iremos usar
covid_ne = covid_ne.iloc[:,[0,2,5,6,11,13,14,26,27,28]].copy() #choose columns
covid_ne.columns = ['id', 'inicioSintomas', 'profissional','cbo', 'resultado', 'sexo', 'estado', 'idade', 'encerramento', 'evolucao']

#usando id como index
covid_ne.index =covid_ne.id
covid_ne.drop(labels="id",axis=1,inplace=True) 



In [ ]:
def formatDate(date_):
  day = date_[0:10]
  day_dt = pd.to_datetime(day,errors='ignore')      
  return day_dt
  



In [ ]:
#como iremos usar em série temporal:
covid_ne.inicioSintomas = covid_ne.inicioSintomas.apply(formatDate)


In [ ]:
covid_ne.sort_values(by=['inicioSintomas'], ascending=False)
covid_ne.loc[covid_ne.inicioSintomas < pd.datetime.now()] #ordenadno por data
covid_ne  = covid_ne.loc[(covid_ne.inicioSintomas <= pd.datetime.now()) & (covid_ne.inicioSintomas >= pd.to_datetime('2020-02-01'))].copy() # eleiminando dados com datas após o dia atual


In [ ]:
#todos os casos de covid por data e estado
covid_grouped = covid_ne.iloc[:,[0,5,8]]
covid_grouped =covid_grouped[['inicioSintomas', 'estado']].groupby(by = ['inicioSintomas'])['estado'].value_counts().unstack().fillna(0)
covid_grouped

In [ ]:
#casos de covid em profissionais de saúde
prof_ne = covid_ne.loc[covid_ne.profissional == 'Sim', ['inicioSintomas', 'estado', 'evolucao']]
prof_ne.estado.unique()


In [ ]:
#novos casos por dia de inicio de sintomas e estado
prof_grouped= prof_ne[['inicioSintomas', 'estado']].groupby(by = ['inicioSintomas'])['estado'].value_counts().unstack().fillna(0)
prof_grouped

In [ ]:
#profissionais que vieram a óbito
prof_obito = prof_ne.loc[ne_date.evolucao=='Óbito',['inicioSintomas', 'estado']]
prof_obito.estado.unique()

In [ ]:
#obitos por estados por dia
ob_grouped = prof_obito.groupby(by = ['inicioSintomas'])['estado'].value_counts().unstack().fillna(0)
ob_grouped['SERGIPE'] = 0.0 #add Sergipe porque de acordo com os dados nenhum profissional de saúde morreu


In [ ]:
ne_states = ['Alagoas', 'Bahia', 'Ceará', 'Maranhão', 'Paraíba', 'Pernambuco', 'Piauí', 'Sergipe', 'Rio Grande do Norte']

In [ ]:
# Using plotly.express
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Initialize figure with subplots
fig = make_subplots(
    rows=3, cols=3, subplot_titles=ne_states, column_widths= [30,30,30], print_grid=True, shared_yaxes=True
)



r = 1
for i, state in enumerate(ne_states):  
  fig.add_scatter(x=prof_grouped.index, y=prof_grouped[ne_states[i].upper()],
              marker=dict(size=2),
              row=r, col=(i%3)+1)
  
  fig.add_scatter(x= ob_grouped.loc[ob_grouped[ne_states[i].upper()] >= 1].index, 
                  y = ob_grouped.loc[ob_grouped[ne_states[i].upper()]>= 1][ne_states[i].upper()],
                  mode="markers",
                  marker=dict(size=5, color="#871414"),
                  row=r, col=(i%3)+1)
  if (((i%3)+1) == 3):
    r += 1


fig.write_html("covid-profissionais-nordeste.html", full)

fig.update_layout(showlegend=False, template= 'plotly_dark')
fig.show()



In [ ]:
help(go.Figure.write_html)
